#### Tensorboard startup

In [1]:
# Load Tensorboard network extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

2023-12-11 15:09:55.329167: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 15:09:55.431529: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 15:09:55.431577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 15:09:55.434913: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 15:09:55.459287: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# clear logs
!rm -rf ./logs/

#### Load Mnist and make model

In [4]:
from tensorflow.keras.layers import Flatten, Dense, Dropout

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

def create_model():
    return tf.keras.models.Sequential([
        Flatten(input_shape=(28,28), name='layers_flatten'),
        Dense(512, activation='relu', name='layers_dense'),
        Dropout(0.2, name='layers_dropout'),
        Dense(10, activation='softmax', name='out_layer')
    ])

#### Model Fitting with Tensorboard

In [6]:
model = create_model()
model.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

logs_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir, histogram_freq=1)

model.fit(
    x=x_train,
    y=y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    callbacks = [tensorboard_callback]
)

Epoch 1/5
1875/1875 [==============================] - 13s 6ms/step - loss: 0.2192 - accuracy: 0.9350 - val_loss: 0.1026 - val_accuracy: 0.9686
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0977 - accuracy: 0.9706 - val_loss: 0.0803 - val_accuracy: 0.9751
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0680 - accuracy: 0.9786 - val_loss: 0.0802 - val_accuracy: 0.9756
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0542 - accuracy: 0.9829 - val_loss: 0.0655 - val_accuracy: 0.9813
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0431 - accuracy: 0.9863 - val_loss: 0.0674 - val_accuracy: 0.9797


In [8]:
# Bind to a localhost port. Makes it easier. Had trouble visualizing with VSCode but this worked just fine
%tensorboard --logdir logs/fit20231211-151054 --host localhost --port 8888

In [9]:
!lsof -i:8888

COMMAND     PID     USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
tensorboa 22827 kmirijan   13u  IPv4 250595      0t0  TCP localhost:8888 (LISTEN)


In [10]:
# Kill tensorboard localhost with the PID above
!kill -9 22827